In [2]:
import pandas as pd
from os import listdir
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
names = ['   ant   ', ' jmeter  ', ' argouml ', ' columba ', '   emf   ', 'hibernate', '  jedit  ', 'jfreechart', '  jruby  ', '  sql12  ']
results = {}
ground_truth = {}
folders = listdir('../../dump_supervised_no_est')
for folder in folders:
    files = listdir('../../dump_supervised_no_est/' + folder)
    for file in files:
        with open('../../dump_supervised_no_est/' + folder + '/' + file, "rb") as handle:
            res = pickle.load(handle)
            results[folder + '__' + file] = res['supervisedQ2']
            ground_truth[file.rsplit('.',1)[0]] = res['true']
dfs = {}
learners = {}
filenames = {}
for file in files:
    df = pd.DataFrame()
    for i, x in results.items():
        if file in i:
            learner = i.split('__')[0]
            filename = file.rsplit('.',1)[0]
            total = ground_truth[filename][1]
            pos = ground_truth[filename][0]
            df[learner + '__cost'] = np.array(list(map(float,x['x'])))/total
            df[learner + '__recall'] = np.array(list(map(float,x['pos'])))/pos
            dfs[filename] = df
            learners[learner] = learner
            filenames[filename] = filename
dfs['apache-ant-1.7.0']

font = {'family': 'normal', 'size': 10}

plt.rc('font', **font)
paras = {'lines.linewidth': 2, 'legend.fontsize': 10, 'axes.labelsize': 10, 'legend.frameon': True,
         'figure.autolayout': False, 'figure.figsize': (16, 12)}
plt.rcParams.update(paras)

fig = plt.figure()

count = 0
count2 = -1
legends = {}
for filename, _ in filenames.items():
    count2 += 1
    count += 1
    if(count==3):
        count +=2
    ax=plt.subplot(3,4,count)
    colors=['red', 'orange', 'brown', 'green', 'blue', 'indigo', 'violet']
    style = ['+', 'd', 'o', 'v', '^', 's', '.']
    count3 = 0
    for learner, _ in learners.items():
        df = dfs[filename]
        x = df[learner + '__cost'].tolist()
        y = df[learner + '__recall'].tolist()
        legend_learner, = ax.plot(x, y, linewidth=2, color=colors[count3], markersize=4, markevery=100, marker=style[count3], linestyle='-',
                            label=learner)
        legends[learner] = legend_learner
        count3 += 1
    ax.annotate(names[count2], (.3,.01), color='k', fontsize=16, verticalalignment='bottom', horizontalalignment='left')
        
    plt.subplots_adjust(left=.1, right=1., wspace = 0.22, hspace = 0.35)
    plt.ylabel("Recall", fontweight='bold', fontsize=14)
    plt.xlabel("Cost", fontweight='bold', fontsize=14)
    
legend_name = []
legends_plot = []
for lgd, lrnr in legends.items():
    legend_name.append(lgd)
    legends_plot.append(lrnr)
plt.figlegend(legends_plot, legend_name, loc=(.7,.66), fontsize=20)
plt.savefig('../../test_figure/'+'no_est_all_learner_retrival'+".png")
plt.savefig('../../test_figure/'+'no_est_all_learner_retrival'+".pdf")
plt.close(fig)
#plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\font_manager.py:1331: UserWarning: findfont: Font family ['normal'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
